In [1]:
import tensorflow as tf
import tensorflow.keras
from keras import Sequential,Model
from keras.layers import SimpleRNN,Input, Lambda, Dense, Flatten, Conv2D, MaxPool2D, Dropout,BatchNormalization,GlobalAvgPool2D
from keras.saving import load_model
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt

In [3]:
cls = Sequential()
cls.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
cls.add(MaxPool2D(pool_size=(2, 2)))
cls.add(Dropout(0.5))
cls.add(Flatten())
cls.add(Dense(128, activation='relu'))
cls.add(Dense(26, activation='softmax'))

In [3]:
cls.load_weights('handwritten6.weights.h5')

In [4]:
import tkinter as tk
from tkinter import ttk,filedialog,messagebox
from ttkbootstrap import Style
import PIL.ImageGrab as ImageGrab
import numpy as np
from tkinter.font import Font

# whiteboard for drawing canvas
class whiteboard:
    def __init__(self,master):
        self.master = master
        self.master.title('Whiteboard')
        self.master.resizable(False,True)
        self.style = Style(theme="pulse")
        self.canvas = tk.Canvas(self.master,width=900,height=720,bg='white')
        self.canvas.pack()
        self.button_frame = ttk.Frame(self.master)
        self.button_frame.pack(side="top",pady=10)
        
        # buttons in the canvas
        button_config = {
            "black": ("dark.TButton",lambda: self.change_color("black")),
            "red": ("danger.TButton",lambda: self.change_color("red")),
            "blue": ("info.TButton",lambda: self.change_color("blue")),
            "green": ("success.TButton",lambda: self.change_color("green")),
            "Eraser": ("light.TButton",self.eraser),
            "clear": ("light.TButton",self.clear_canvas),
            "Convert": ("light.TButton",self.recognize),
        }
        
        for color,(style,command) in button_config.items():
            ttk.Button(self.button_frame,text=color.capitalize(),
                      command=command,style=style).pack(side='left',padx=5,pady=5)
        
        self.draw_color = "black"
        self.line_width = 14
        self.old_x, self.old_y = None,None
        
        self.canvas.bind("<Button-1>",self.start_line)
        self.canvas.bind("<B1-Motion>",self.draw_line)
        
        self.root1 = tk.Tk()
        self.brp = self.notepad(self.root1)
        self.root1.mainloop()
        
    def start_line(self,event):
        self.old_x , self.old_y = event.x , event.y
        
    # drawing on the canvas
    def draw_line(self,event):
        if self.old_x and self.old_y:
            self.canvas.create_line(self.old_x , self.old_y, event.x, event.y,
                                   width=self.line_width, fill=self.draw_color,
                                   capstyle=tk.ROUND, smooth = tk.TRUE)
            self.old_x,self.old_y = event.x, event.y
    
    # changing color of the pen
    def change_color(self,new_color):
        self.draw_color = new_color
        self.line_width = 14
        
    # clear canvas content
    def clear_canvas(self):
        if messagebox.askokcancel('clear screen','Are you sure'):
            self.canvas.delete("all")
        else:
            pass
        
    # eraser
    def eraser(self):
        self.draw_color = "White"
        self.line_width = 15
     
    # converting image to text
    def recognize(self):
        try:
            x=self.master.winfo_rootx()
            y=self.master.winfo_rooty()
            img = ImageGrab.grab(bbox=(x+5,y+22,x+1100,y+900))
            img_arr = np.array(img)
            self.find_sentence(img_arr)
            self.canvas.delete("all")
        except Exception as e:
            print(e)
        
    def insert(self):
        self.brp.insert_note('Animesh')
      
    # notepad 
    class notepad:
        def __init__(self,master):
            self.master1 = master
            self.master1.title("Text Recognizer")
            self.master1.geometry("644x700")
            self.TextArea = tk.Text(self.master1, font=Font(family="lucida 13",size=38))
            self.file = None
            self.TextArea.pack(expand=True, fill=tk.BOTH)
            self.MenuBar = tk.Menu(self.master1)
            self.FileMenu = tk.Menu(self.MenuBar, tearoff=0)
            self.FileMenu.add_command(label="New", command=self.newFile)
            self.FileMenu.add_separator()
            self.FileMenu.add_command(label = "Exit", command=self.quitApp)
            self.MenuBar.add_cascade(label = "File", menu=self.FileMenu)
            self.master1.config(menu=self.MenuBar)


        def newFile(self):
            global file
            self.master1.title("Text Recognizer")
            self.file = None
            self.TextArea.delete(1.0, tk.END)

        def quitApp(self):
            self.master1.destroy()
        
        def insert_note(self,texxt):
            self.TextArea.insert(tk.INSERT, texxt)
    
    # find individual sentence in sequence
    def find_sentence(self,IImage):    
        sentence = []
        img = cv2.cvtColor(IImage,cv2.COLOR_BGR2GRAY)
        imgFiltered1 = cv2.filter2D(img, -1, self.createKernel(15, 11, 7), borderType=cv2.BORDER_REPLICATE)
        (_, imgThres) = cv2.threshold(imgFiltered1, 150, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        imgThres = 255 - imgThres
        rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (120,8))
        imgThres = cv2.dilate(imgThres, rect_kernel, iterations = 1)
        (components, _) = cv2.findContours(imgThres, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contour = components
        for cnt in contour:
            x, y, w, h = cv2.boundingRect(cnt)
            sentence.append(img[y:y+h, x:x+w])
        sentence.reverse()
        self.find_words(sentence)
        
    def createKernel(self,kernelSize, sigma, theta):
        assert kernelSize % 2 # must be odd size
        halfSize = kernelSize // 2

        kernel = np.zeros([kernelSize, kernelSize])
        sigmaX = sigma
        sigmaY = sigma * theta

        for i in range(kernelSize):
            for j in range(kernelSize):
                x = i - halfSize
                y = j - halfSize

                expTerm = np.exp(-x**2 / (2 * sigmaX) - y**2 / (2 * sigmaY))
                xTerm = (x**2 - sigmaX**2) / (2 * math.pi * sigmaX**5 * sigmaY)
                yTerm = (y**2 - sigmaY**2) / (2 * math.pi * sigmaY**5 * sigmaX)

                kernel[i, j] = (xTerm + yTerm) * expTerm

        kernel = kernel / np.sum(kernel)
        return kernel
    
    # find individual words in the sentence
    def find_words(self,rese):
        words=[]
        for result in rese:
            temp=[]
            imgFiltered1 = cv2.filter2D(result, -1, self.createKernel(23, 11, 7), borderType=cv2.BORDER_REPLICATE)
            (_, imgThres) = cv2.threshold(imgFiltered1, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            imgThres = 255 - imgThres
            rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20,8))
            imgThres = cv2.dilate(imgThres, rect_kernel, iterations = 1)
            (components, _) = cv2.findContours(imgThres, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            for cnt in components:
                x, y, w, h = cv2.boundingRect(cnt)
                if cv2.contourArea(cnt) < 300:
                    continue
                coordinate = (x,y)
                currImg = result[y:y+h, x:x+w]
                temp.append((coordinate, currImg))
            if len(words)==0:
                words.append(sorted(temp, key=lambda entry:entry[0][0]))
            else:
                temp = sorted(temp, key=lambda entry:entry[0][0])
                for i in range(len(temp)):
                    words[0].append(temp[i])
     #   rump.reverse()
        self.individual_letter(words)
        
    # find individual letter in the word
    def individual_letter(self,rump_arr):
        alphabet = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
        word=[]

        for i in range(len(rump_arr[0])):
            temp = []
            raga = []
            img = rump_arr[0][i][1]
            (_, imgThres) = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY_INV)
            contours, hierarchy = cv2.findContours(imgThres, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            for cnt in contours:
                x, y, w, h = cv2.boundingRect(cnt)
                if cv2.contourArea(cnt) < 50:
                    continue
                coordinate = (x,y)
                letter = img[y:y+h, x:x+w]
                temp.append((coordinate, letter))
            if len(raga)==0:
                raga.append(sorted(temp, key=lambda entry:entry[0][0]))
            else:
                temp = sorted(temp, key=lambda entry:entry[0][0])
                for i in range(len(temp)):
                    raga[0].append(temp[i])
            for group in raga[0]:
                image = group[1]
                arg = cv2.resize(255-image,(28,28))
                arg = arg.astype("float32") / 255.0
                para = arg.reshape(1,28,28,1)
                pred = np.argmax(cls.predict(para))
                word.append(alphabet[pred])
            word.append(" ")
        final_word = "".join(word)
        self.brp.insert_note(final_word)
            
    

root = tk.Tk()
whiteboard = whiteboard(root)
root.mainloop()